# KubeVirt

KubeVirt ist ein Open-Source-Projekt, das es ermöglicht, virtuelle Maschinen auf Kubernetes bereitzustellen, zu nutzen und zu verwalten – ähnlich wie Container. 

Durch eine einheitliche Bereitstellungs- und Verwaltungsebene für sowohl Container als auch virtuelle Maschinen entsteht eine vereinheitlichte Plattform für cloud-native Anwendungen, unabhängig von den Anforderungen.

Quelle: Unterhaltung mit Copilot, 18.8.2024
* (1) Getting Started with KubeVirt Containers and Virtual Machines Together. https://www.redhat.com/en/blog/getting-started-with-kubevirt.
* (2) Introduction | KubeVirt.io. https://kubevirt.io/docs/.
* (3) GitHub - kubevirt/kubevirt: Kubernetes Virtualization API and runtime .... https://github.com/kubevirt/kubevirt.
* (4) Overview of KubeVirt | Kube by Example. https://kubebyexample.com/learning-paths/kubevirt-fundamentals/overview-kubevirt.

### Validate Hardware Virtualization Support
Hardware with virtualization support is recommended. You can use virt-host-validate to ensure that your hosts are capable of running virtualization workloads:

In [ ]:
! sudo usermod -aG kvm $USER
! sudo chmod 666 /dev/kvm


In [ ]:
! virt-host-validate qemu

Und Installieren KubeVirt mittels Operators

In [ ]:
%%bash
export VERSION=$(curl -s https://storage.googleapis.com/kubevirt-prow/release/kubevirt/kubevirt/stable.txt)
echo $VERSION

kubectl apply -f https://github.com/kubevirt/kubevirt/releases/download/${VERSION}/kubevirt-operator.yaml
kubectl apply -f https://github.com/kubevirt/kubevirt/releases/download/${VERSION}/kubevirt-cr.yaml

# kubectl -n kubevirt wait kv kubevirt --for condition=Available

# enabling nested virtualization
kubectl -n kubevirt patch kubevirt kubevirt --type=merge --patch '{"spec":{"configuration":{"developerConfiguration":{"useEmulation":true}}}}'

Kontrollieren, bzw. warten bis alles läuft.

Es sollten 6 Pods laufen

In [ ]:
! kubectl get kubevirt.kubevirt.io/kubevirt -n kubevirt -o=jsonpath="{.status.phase}"
! kubectl get pods -n kubevirt

Wir brauchen zusätzlich das CLI `virtctl` um VMs zu starten, stoppen oder sich mit diesen zu verbinden:

In [ ]:
%%bash
export VERSION=$(curl -s https://api.github.com/repos/kubevirt/kubevirt/releases/latest | jq -r .tag_name)
echo ${VERSION}
wget https://github.com/kubevirt/kubevirt/releases/download/$VERSION/virtctl-$VERSION-linux-amd64
chmod +x virtctl-$VERSION-linux-amd64
sudo mv virtctl-$VERSION-linux-amd64 /usr/local/bin/virtctl
virtctl

- - -

### Erstellen von VMs

Zuerst brauchen wir eine Deklaration im YAML Format

In [ ]:
%%bash
cat <<%EOF% | kubectl apply -f - 
apiVersion: kubevirt.io/v1
kind: VirtualMachine
metadata:
  name: ubuntu-vm
  namespace: default
spec:
  running: false
  template:
    metadata:
      labels:
        kubevirt.io/domain: ubuntu-vm
    spec:
      domain:
        cpu:
          cores: 2
        devices:
          disks:
          - disk:
              bus: virtio
            name: containerdisk
          - disk:
              bus: virtio
            name: cloudinitdisk
          interfaces:  # Netzwerkschnittstellen hier definieren
          - name: default
            masquerade: {}  # Verwende das Masquerade-Netzwerk für NAT
        memory:
          guest: 4Gi
      networks:
      - name: default
        pod: {}  # Verweist auf das Standard-Pod-Netzwerk
      volumes:
      - name: containerdisk
        containerDisk:
          image: quay.io/containerdisks/ubuntu:24.04
      - name: cloudinitdisk
        cloudInitNoCloud:
          userData: |
            #cloud-config
            users:
              - name: ubuntu
                passwd: insecure
                lock_passwd: false
                sudo: ALL=(ALL) NOPASSWD:ALL
                ssh_pwauth: true
            ssh_pwauth: true
%EOF%


In [ ]:
! virtctl start ubuntu-vm

In [ ]:
! kubectl get vm

- - -

### Aufräumen

In [ ]:
! kind delete cluster 
! kubectl config use-context microk8s